<!-- _Note: this step seems to use up all of my 16 GB of RAM, so it's very possible that it won't run as-is with less than 16 GB. Processing of HRRR data is done in approximately 1-year chunks - splitting this up into, e.g., 3- or 6-month chunks would likely speed things up and reduce the RAM requirements. Code in later sections would have to be adjusted accordingly to use the new collection of netCDF .nc files._ -->

In [1]:
import pandas as pd
from solar_forecaster.herbie_gefs_tools import get_many_tcc_fcasts_fastherbie
from solar_forecaster.herbie_hrrr_tools import get_many_hrrr_fcasts_fastherbie
import gc

Load plant specifications:

In [2]:
plants_df = pd.read_csv('pv_fleet_metadata_rev3.csv') # read in as a Pandas DataFrame
plants_df = plants_df[plants_df['ac_capacity']>=10].copy().reset_index()
num_plants = len(plants_df.index) # number of plants

Get GEFS data:

In [3]:
gc.collect() 

query_len_days = 10

num_members = 30 # Number of ensemble members to pull, max of 30
freq='1D' # how often to pull a new forecast
fxx_interval = 3 # time step of the forecast, GEFS has 3 hour intervals
fxx_range = range(27,46,fxx_interval)

# lat, lon extent for region of interest
# (left lon, right lon, lower lat, upper lat)
region_extent = [-92, -80, 29, 36]

start = '2020-12-31T06:00:00'
end = '2024-01-01T06:00'

save_dir = 'nc_files/gefs/'
remove_grib=True
w_avg_tcc = get_many_tcc_fcasts_fastherbie(start, end, freq, query_len_days,
                                        fxx_range, plants_df, region_extent,
                                        num_members=num_members,
                                        save=True, save_dir=save_dir,
                                        remove_grib=remove_grib)

# clean up indices and localize to UTC
w_avg_tcc = w_avg_tcc.reset_index().set_index('valid_time_utc_end_of_interval').tz_localize('UTC')

# save as pickle file
w_avg_tcc.to_pickle('data_files/w_avg_tcc_2021-2023.pkl')

loaded gefs_total_cloud_cover_f27-f45_3h_2020123106-2021010906_1D_step_30_members.nc
loaded gefs_total_cloud_cover_f27-f45_3h_2021011006-2021011906_1D_step_30_members.nc
loaded gefs_total_cloud_cover_f27-f45_3h_2021012006-2021012906_1D_step_30_members.nc
loaded gefs_total_cloud_cover_f27-f45_3h_2021013006-2021020806_1D_step_30_members.nc
loaded gefs_total_cloud_cover_f27-f45_3h_2021020906-2021021806_1D_step_30_members.nc
loaded gefs_total_cloud_cover_f27-f45_3h_2021021906-2021022806_1D_step_30_members.nc
loaded gefs_total_cloud_cover_f27-f45_3h_2021030106-2021031006_1D_step_30_members.nc
loaded gefs_total_cloud_cover_f27-f45_3h_2021031106-2021032006_1D_step_30_members.nc
loaded gefs_total_cloud_cover_f27-f45_3h_2021032106-2021033006_1D_step_30_members.nc
loaded gefs_total_cloud_cover_f27-f45_3h_2021033106-2021040906_1D_step_30_members.nc
loaded gefs_total_cloud_cover_f27-f45_3h_2021041006-2021041906_1D_step_30_members.nc
loaded gefs_total_cloud_cover_f27-f45_3h_2021042006-2021042906_1D

In [4]:
w_avg_tcc.head()

,level_0,time_utc,step,tcc_std,tcc_p01,tcc_p02,tcc_p03,tcc_p04,tcc_p05,tcc_p06,...,tcc_p21,tcc_p22,tcc_p23,tcc_p24,tcc_p25,tcc_p26,tcc_p27,tcc_p28,tcc_p29,tcc_p30
valid_time_utc_end_of_interval,,,,,,,,,,,,,,,,,,,,,
2021-01-01 07:00:00+00:00,0,2020-12-31 06:00:00,1 days 03:00:00,0.657303,98.406453,99.825122,99.723112,99.455641,98.431235,99.197912,...,98.459086,98.447650,98.465006,99.460350,99.642682,98.839168,99.351640,98.563921,99.583995,98.303555
2021-01-01 08:00:00+00:00,0,2020-12-31 06:00:00,1 days 03:00:00,0.657303,98.406453,99.825122,99.723112,99.455641,98.431235,99.197912,...,98.459086,98.447650,98.465006,99.460350,99.642682,98.839168,99.351640,98.563921,99.583995,98.303555
2021-01-01 09:00:00+00:00,0,2020-12-31 06:00:00,1 days 03:00:00,0.657303,98.406453,99.825122,99.723112,99.455641,98.431235,99.197912,...,98.459086,98.447650,98.465006,99.460350,99.642682,98.839168,99.351640,98.563921,99.583995,98.303555
2021-01-01 10:00:00+00:00,0,2020-12-31 06:00:00,1 days 06:00:00,1.346134,97.210989,99.084062,98.274144,98.806474,96.843043,99.043834,...,97.420095,97.805882,98.418992,98.181498,97.591987,98.777332,99.539623,94.137960,99.059683,97.906520
2021-01-01 11:00:00+00:00,0,2020-12-31 06:00:00,1 days 06:00:00,1.346134,97.210989,99.084062,98.274144,98.806474,96.843043,99.043834,...,97.420095,97.805882,98.418992,98.181498,97.591987,98.777332,99.539623,94.137960,99.059683,97.906520


Get HRRR actuals (F00) for 2021, 2022, and 2023. We will get each year separately due to memory constraints with processing the xarray datasets. Running `gc.collect()` in between each year may help with memory issues. 

In [5]:
gc.collect()

start = '2021-01-01 06:00'
end = '2022-01-02 05:00'

freq = '1h'
fxx_range = range(0,1)
query_len_days = 6
save_dir='nc_files/hrrr/'
ds, p_ac_all, p_ac_cs_all = get_many_hrrr_fcasts_fastherbie(start, end, freq, query_len_days, fxx_range,
                                    plants_df, region_extent=[220,440,1080,1440],
                                    save=True, save_dir=save_dir, attempts=5,
                                    transposition_model='perez',
                                    decomposition_model='erbs')

p_ac_all = p_ac_all.reset_index().set_index('time_center_labeled')
p_ac_cs_all = p_ac_cs_all.reset_index().set_index('time_center_labeled')

ds.to_netcdf('nc_files/ds_act_2021.nc')
p_ac_all.to_pickle('data_files/p_ac_all_act_2021.pkl')

loaded hrrr_solar_f0-f0_1h_2021010106-2021010705_1h_step.nc
loaded hrrr_solar_f0-f0_1h_2021010706-2021011305_1h_step.nc
loaded hrrr_solar_f0-f0_1h_2021011306-2021011905_1h_step.nc
loaded hrrr_solar_f0-f0_1h_2021011906-2021012505_1h_step.nc
loaded hrrr_solar_f0-f0_1h_2021012506-2021013105_1h_step.nc
loaded hrrr_solar_f0-f0_1h_2021013106-2021020605_1h_step.nc
loaded hrrr_solar_f0-f0_1h_2021020606-2021021205_1h_step.nc
loaded hrrr_solar_f0-f0_1h_2021021206-2021021805_1h_step.nc
loaded hrrr_solar_f0-f0_1h_2021021806-2021022405_1h_step.nc
loaded hrrr_solar_f0-f0_1h_2021022406-2021030205_1h_step.nc
loaded hrrr_solar_f0-f0_1h_2021030206-2021030805_1h_step.nc
loaded hrrr_solar_f0-f0_1h_2021030806-2021031405_1h_step.nc
loaded hrrr_solar_f0-f0_1h_2021031406-2021032005_1h_step.nc
loaded hrrr_solar_f0-f0_1h_2021032006-2021032605_1h_step.nc
loaded hrrr_solar_f0-f0_1h_2021032606-2021040105_1h_step.nc
loaded hrrr_solar_f0-f0_1h_2021040106-2021040705_1h_step.nc
loaded hrrr_solar_f0-f0_1h_2021040706-20

In [7]:
gc.collect()

start = '2022-01-02 06:00'
end = '2023-01-03 05:00'

freq = '1h'
fxx_range = range(0,1)
query_len_days = 6
save_dir='nc_files/hrrr/'
ds, p_ac_all, p_ac_cs_all = get_many_hrrr_fcasts_fastherbie(start, end, freq, query_len_days, fxx_range,
                                    plants_df, region_extent=[220,440,1080,1440],
                                    save=True, save_dir=save_dir, attempts=5,
                                    transposition_model='perez',
                                    decomposition_model='erbs')

p_ac_all = p_ac_all.reset_index().set_index('time_center_labeled')
p_ac_cs_all = p_ac_cs_all.reset_index().set_index('time_center_labeled')

ds.to_netcdf('nc_files/ds_act_2022.nc')
p_ac_all.to_pickle('data_files/p_ac_all_act_2022.pkl')

loaded hrrr_solar_f0-f0_1h_2022010206-2022010805_1h_step.nc
loaded hrrr_solar_f0-f0_1h_2022010806-2022011405_1h_step.nc
loaded hrrr_solar_f0-f0_1h_2022011406-2022012005_1h_step.nc
loaded hrrr_solar_f0-f0_1h_2022012006-2022012605_1h_step.nc
loaded hrrr_solar_f0-f0_1h_2022012606-2022020105_1h_step.nc
loaded hrrr_solar_f0-f0_1h_2022020106-2022020705_1h_step.nc
loaded hrrr_solar_f0-f0_1h_2022020706-2022021305_1h_step.nc
loaded hrrr_solar_f0-f0_1h_2022021306-2022021905_1h_step.nc
loaded hrrr_solar_f0-f0_1h_2022021906-2022022505_1h_step.nc
loaded hrrr_solar_f0-f0_1h_2022022506-2022030305_1h_step.nc
loaded hrrr_solar_f0-f0_1h_2022030306-2022030905_1h_step.nc
loaded hrrr_solar_f0-f0_1h_2022030906-2022031505_1h_step.nc
loaded hrrr_solar_f0-f0_1h_2022031506-2022032105_1h_step.nc
loaded hrrr_solar_f0-f0_1h_2022032106-2022032705_1h_step.nc
loaded hrrr_solar_f0-f0_1h_2022032706-2022040205_1h_step.nc
loaded hrrr_solar_f0-f0_1h_2022040206-2022040805_1h_step.nc
loaded hrrr_solar_f0-f0_1h_2022040806-20

In [5]:
gc.collect()

start = '2023-01-03 06:00'
end = '2024-01-04 05:00'

freq = '1h'
fxx_range = range(0,1)
query_len_days = 6
save_dir='nc_files/hrrr/'
ds, p_ac_all, p_ac_cs_all = get_many_hrrr_fcasts_fastherbie(start, end, freq, query_len_days, fxx_range,
                                    plants_df, region_extent=[220,440,1080,1440],
                                    save=True, save_dir=save_dir, attempts=5,
                                    transposition_model='perez',
                                    decomposition_model='erbs')

p_ac_all = p_ac_all.reset_index().set_index('time_center_labeled')
p_ac_cs_all = p_ac_cs_all.reset_index().set_index('time_center_labeled')

ds.to_netcdf('nc_files/ds_act_2023.nc')
p_ac_all.to_pickle('data_files/p_ac_all_act_2023.pkl')

loaded hrrr_solar_f0-f0_1h_2023010306-2023010905_1h_step.nc
loaded hrrr_solar_f0-f0_1h_2023010906-2023011505_1h_step.nc
loaded hrrr_solar_f0-f0_1h_2023011506-2023012105_1h_step.nc
loaded hrrr_solar_f0-f0_1h_2023012106-2023012705_1h_step.nc
loaded hrrr_solar_f0-f0_1h_2023012706-2023020205_1h_step.nc
loaded hrrr_solar_f0-f0_1h_2023020206-2023020805_1h_step.nc
loaded hrrr_solar_f0-f0_1h_2023020806-2023021405_1h_step.nc
loaded hrrr_solar_f0-f0_1h_2023021406-2023022005_1h_step.nc
loaded hrrr_solar_f0-f0_1h_2023022006-2023022605_1h_step.nc
loaded hrrr_solar_f0-f0_1h_2023022606-2023030405_1h_step.nc
loaded hrrr_solar_f0-f0_1h_2023030406-2023031005_1h_step.nc
loaded hrrr_solar_f0-f0_1h_2023031006-2023031605_1h_step.nc
loaded hrrr_solar_f0-f0_1h_2023031606-2023032205_1h_step.nc
loaded hrrr_solar_f0-f0_1h_2023032206-2023032805_1h_step.nc
loaded hrrr_solar_f0-f0_1h_2023032806-2023040305_1h_step.nc
loaded hrrr_solar_f0-f0_1h_2023040306-2023040905_1h_step.nc
loaded hrrr_solar_f0-f0_1h_2023040906-20

Similarly, get HRRR day ahead forecasts for 2021, 2022, and 2023:

In [6]:
start = '2020-12-31T06:00:00'
end = '2021-12-31T06:00'

freq = '24h'
fxx_range = range(27,45)
query_len_days = 3

save_dir = 'nc_files/hrrr/'
remove_grib=True

# coarsened mean
gc.collect()
ds, p_ac_all, p_ac_cs_all = get_many_hrrr_fcasts_fastherbie(start, end, freq, query_len_days, fxx_range,
                                    plants_df, region_extent=[220,440,1080,1440],
                                    save=True, save_dir=save_dir, attempts=5,
                                    transposition_model='perez',
                                    decomposition_model='erbs',
                                    coarsen_type='mean', window_size=10)
ds.to_netcdf('nc_files/ds_fcast_2021.nc')
ds.close()
del ds
p_ac_all = p_ac_all.reset_index().set_index('time_center_labeled').drop(columns={'level_0'})
p_ac_all.to_pickle('data_files/p_ac_all_fcast_2021.pkl')

# save a clear sky profile 
p_ac_cs_all = p_ac_cs_all.reset_index().set_index('time_center_labeled').drop(columns={'level_0'})
p_ac_cs_all.to_pickle('data_files/p_ac_all_cs_2021.pkl')

# coarsened max
gc.collect()
ds, p_ac_all, p_ac_cs_all = get_many_hrrr_fcasts_fastherbie(start, end, freq, query_len_days, fxx_range,
                                    plants_df, region_extent=[220,440,1080,1440],
                                    save=True, save_dir=save_dir, attempts=5,
                                    transposition_model='perez',
                                    decomposition_model='erbs',
                                    coarsen_type='max', window_size=10)
ds.to_netcdf('nc_files/ds_fcast_max_2021.nc')
ds.close()
del ds
p_ac_all = p_ac_all.reset_index().set_index('time_center_labeled').drop(columns={'level_0'})
p_ac_all.to_pickle('data_files/p_ac_all_fcast_max_2021.pkl')

# coarsened min
gc.collect()
ds, p_ac_all, p_ac_cs_all = get_many_hrrr_fcasts_fastherbie(start, end, freq, query_len_days, fxx_range,
                                    plants_df, region_extent=[220,440,1080,1440],
                                    save=True, save_dir=save_dir, attempts=5,
                                    transposition_model='perez',
                                    decomposition_model='erbs',
                                    coarsen_type='min', window_size=10)
ds.to_netcdf('nc_files/ds_fcast_min_2021.nc')
ds.close()
del ds
p_ac_all = p_ac_all.reset_index().set_index('time_center_labeled').drop(columns={'level_0'})
p_ac_all.to_pickle('data_files/p_ac_all_fcast_min_2021.pkl')

loaded hrrr_solar_f27-f44_1h_2020123106-2021010206_24h_step.nc
loaded hrrr_solar_f27-f44_1h_2021010306-2021010506_24h_step.nc
loaded hrrr_solar_f27-f44_1h_2021010606-2021010806_24h_step.nc
loaded hrrr_solar_f27-f44_1h_2021010906-2021011106_24h_step.nc
loaded hrrr_solar_f27-f44_1h_2021011206-2021011406_24h_step.nc
loaded hrrr_solar_f27-f44_1h_2021011506-2021011706_24h_step.nc
loaded hrrr_solar_f27-f44_1h_2021011806-2021012006_24h_step.nc
loaded hrrr_solar_f27-f44_1h_2021012106-2021012306_24h_step.nc
loaded hrrr_solar_f27-f44_1h_2021012406-2021012606_24h_step.nc
loaded hrrr_solar_f27-f44_1h_2021012706-2021012906_24h_step.nc
loaded hrrr_solar_f27-f44_1h_2021013006-2021020106_24h_step.nc
loaded hrrr_solar_f27-f44_1h_2021020206-2021020406_24h_step.nc
loaded hrrr_solar_f27-f44_1h_2021020506-2021020706_24h_step.nc
loaded hrrr_solar_f27-f44_1h_2021020806-2021021006_24h_step.nc
loaded hrrr_solar_f27-f44_1h_2021021106-2021021306_24h_step.nc
loaded hrrr_solar_f27-f44_1h_2021021406-2021021606_24h_

In [8]:
start = '2022-01-01T06:00:00'
end = '2023-01-01T06:00'

freq = '24h'
fxx_range = range(27,45)
query_len_days = 3

save_dir = 'nc_files/hrrr/'
remove_grib=True

# coarsened mean
gc.collect()
ds, p_ac_all, p_ac_cs_all = get_many_hrrr_fcasts_fastherbie(start, end, freq, query_len_days, fxx_range,
                                    plants_df, region_extent=[220,440,1080,1440],
                                    save=True, save_dir=save_dir, attempts=5,
                                    transposition_model='perez',
                                    decomposition_model='erbs',
                                    coarsen_type='mean', window_size=10)
ds.to_netcdf('nc_files/ds_fcast_2022.nc')
ds.close()
del ds
p_ac_all = p_ac_all.reset_index().set_index('time_center_labeled').drop(columns={'level_0'})
p_ac_all.to_pickle('data_files/p_ac_all_fcast_2022.pkl')

# save a clear sky profile 
p_ac_cs_all = p_ac_cs_all.reset_index().set_index('time_center_labeled').drop(columns={'level_0'})
p_ac_cs_all.to_pickle('data_files/p_ac_all_cs_2022.pkl')

# coarsened max
gc.collect()
ds, p_ac_all, p_ac_cs_all = get_many_hrrr_fcasts_fastherbie(start, end, freq, query_len_days, fxx_range,
                                    plants_df, region_extent=[220,440,1080,1440],
                                    save=True, save_dir=save_dir, attempts=5,
                                    transposition_model='perez',
                                    decomposition_model='erbs',
                                    coarsen_type='max', window_size=10)
ds.to_netcdf('nc_files/ds_fcast_max_2022.nc')
ds.close()
del ds
p_ac_all = p_ac_all.reset_index().set_index('time_center_labeled').drop(columns={'level_0'})
p_ac_all.to_pickle('data_files/p_ac_all_fcast_max_2022.pkl')

# coarsened min
gc.collect()
ds, p_ac_all, p_ac_cs_all = get_many_hrrr_fcasts_fastherbie(start, end, freq, query_len_days, fxx_range,
                                    plants_df, region_extent=[220,440,1080,1440],
                                    save=True, save_dir=save_dir, attempts=5,
                                    transposition_model='perez',
                                    decomposition_model='erbs',
                                    coarsen_type='min', window_size=10)
ds.to_netcdf('nc_files/ds_fcast_min_2022.nc')
ds.close()
del ds
p_ac_all = p_ac_all.reset_index().set_index('time_center_labeled').drop(columns={'level_0'})
p_ac_all.to_pickle('data_files/p_ac_all_fcast_min_2022.pkl')

loaded hrrr_solar_f27-f44_1h_2022010106-2022010306_24h_step.nc
loaded hrrr_solar_f27-f44_1h_2022010406-2022010606_24h_step.nc
loaded hrrr_solar_f27-f44_1h_2022010706-2022010906_24h_step.nc
loaded hrrr_solar_f27-f44_1h_2022011006-2022011206_24h_step.nc
loaded hrrr_solar_f27-f44_1h_2022011306-2022011506_24h_step.nc
loaded hrrr_solar_f27-f44_1h_2022011606-2022011806_24h_step.nc
loaded hrrr_solar_f27-f44_1h_2022011906-2022012106_24h_step.nc
loaded hrrr_solar_f27-f44_1h_2022012206-2022012406_24h_step.nc
loaded hrrr_solar_f27-f44_1h_2022012506-2022012706_24h_step.nc
loaded hrrr_solar_f27-f44_1h_2022012806-2022013006_24h_step.nc
loaded hrrr_solar_f27-f44_1h_2022013106-2022020206_24h_step.nc
loaded hrrr_solar_f27-f44_1h_2022020306-2022020506_24h_step.nc
loaded hrrr_solar_f27-f44_1h_2022020606-2022020806_24h_step.nc
loaded hrrr_solar_f27-f44_1h_2022020906-2022021106_24h_step.nc
loaded hrrr_solar_f27-f44_1h_2022021206-2022021406_24h_step.nc
loaded hrrr_solar_f27-f44_1h_2022021506-2022021706_24h_

In [9]:
start = '2023-01-02T06:00:00'
end = '2024-01-01T06:00'

freq = '24h'
fxx_range = range(27,45)
query_len_days = 3

save_dir = 'nc_files/hrrr/'
remove_grib=True

# coarsened mean
gc.collect()
ds, p_ac_all, p_ac_cs_all = get_many_hrrr_fcasts_fastherbie(start, end, freq, query_len_days, fxx_range,
                                    plants_df, region_extent=[220,440,1080,1440],
                                    save=True, save_dir=save_dir, attempts=5,
                                    transposition_model='perez',
                                    decomposition_model='erbs',
                                    coarsen_type='mean', window_size=10)
ds.to_netcdf('nc_files/ds_fcast_2023.nc')
ds.close()
del ds
p_ac_all = p_ac_all.reset_index().set_index('time_center_labeled').drop(columns={'level_0'})
p_ac_all.to_pickle('data_files/p_ac_all_fcast_2023.pkl')

# save a clear sky profile 
p_ac_cs_all = p_ac_cs_all.reset_index().set_index('time_center_labeled').drop(columns={'level_0'})
p_ac_cs_all.to_pickle('data_files/p_ac_all_cs_2023.pkl')

# coarsened max
gc.collect()
ds, p_ac_all, p_ac_cs_all = get_many_hrrr_fcasts_fastherbie(start, end, freq, query_len_days, fxx_range,
                                    plants_df, region_extent=[220,440,1080,1440],
                                    save=True, save_dir=save_dir, attempts=5,
                                    transposition_model='perez',
                                    decomposition_model='erbs',
                                    coarsen_type='max', window_size=10)
ds.to_netcdf('nc_files/ds_fcast_max_2023.nc')
ds.close()
del ds
p_ac_all = p_ac_all.reset_index().set_index('time_center_labeled').drop(columns={'level_0'})
p_ac_all.to_pickle('data_files/p_ac_all_fcast_max_2023.pkl')

# coarsened min
gc.collect()
ds, p_ac_all, p_ac_cs_all = get_many_hrrr_fcasts_fastherbie(start, end, freq, query_len_days, fxx_range,
                                    plants_df, region_extent=[220,440,1080,1440],
                                    save=True, save_dir=save_dir, attempts=5,
                                    transposition_model='perez',
                                    decomposition_model='erbs',
                                    coarsen_type='min', window_size=10)
ds.to_netcdf('nc_files/ds_fcast_min_2023.nc')
ds.close()
del ds
p_ac_all = p_ac_all.reset_index().set_index('time_center_labeled').drop(columns={'level_0'})
p_ac_all.to_pickle('data_files/p_ac_all_fcast_min_2023.pkl')

loaded hrrr_solar_f27-f44_1h_2023010206-2023010406_24h_step.nc
loaded hrrr_solar_f27-f44_1h_2023010506-2023010706_24h_step.nc
loaded hrrr_solar_f27-f44_1h_2023010806-2023011006_24h_step.nc
loaded hrrr_solar_f27-f44_1h_2023011106-2023011306_24h_step.nc
loaded hrrr_solar_f27-f44_1h_2023011406-2023011606_24h_step.nc
loaded hrrr_solar_f27-f44_1h_2023011706-2023011906_24h_step.nc
loaded hrrr_solar_f27-f44_1h_2023012006-2023012206_24h_step.nc
loaded hrrr_solar_f27-f44_1h_2023012306-2023012506_24h_step.nc
loaded hrrr_solar_f27-f44_1h_2023012606-2023012806_24h_step.nc
loaded hrrr_solar_f27-f44_1h_2023012906-2023013106_24h_step.nc
loaded hrrr_solar_f27-f44_1h_2023020106-2023020306_24h_step.nc
loaded hrrr_solar_f27-f44_1h_2023020406-2023020606_24h_step.nc
loaded hrrr_solar_f27-f44_1h_2023020706-2023020906_24h_step.nc
loaded hrrr_solar_f27-f44_1h_2023021006-2023021206_24h_step.nc
loaded hrrr_solar_f27-f44_1h_2023021306-2023021506_24h_step.nc
loaded hrrr_solar_f27-f44_1h_2023021606-2023021806_24h_